In [11]:
# Import required packages 
import os
import glob
import cv2
import numpy as np
import pandas as pd
import random

# Read the training data file with error handling for bad lines
# data = pd.read_excel("D:\\updated_final_book_1.xlsx")  
data = pd.read_excel("/Users/almazbaghirova/Desktop/Hiwi/VideoCut/test_videos.xlsx")


# Display the first few rows
print(data.head())
print(list(data.columns))
data.info()


def convert_path(path):
    # Remove 'file://' prefix if present
    if path.startswith('file://'):
        path = path[7:]
    # Normalize the path and ensure it's in the correct format
    path = os.path.normpath(path)
    return path

# Check dimensions of the subset 
print(data.shape)
data['HighRes_Drive_Path'] = data['HighRes_Drive_Path'].apply(lambda x: convert_path(x.strip()))


        Date  Obs_no Flight  Total_flights  Start_time  End_time    Drone  \
0 2023-08-07       1      A              1         NaN       NaN  Pink_M3   
1 2023-08-07       1      A              1         NaN       NaN  Pink_M3   
2 2023-08-07       1      A              1         NaN       NaN  Pink_M3   
3 2023-08-07       1      A              1         NaN       NaN  Pink_M3   

  Pilot Location     Video  ... group size start   end  light quality   \
0    AD     GCP1  DJI_0404  ...        NaN  10.0  30.0             NaN   
1    AD     GCP1  DJI_0405  ...        NaN  20.0  40.0             NaN   
2    AD     GCP1  DJI_0406  ...        NaN   NaN   NaN             NaN   
3    AD     GCP1  DJI_0407  ...        NaN   NaN   NaN             NaN   

   habitat                      unique_id  \
0      NaN  0708_001_PM3_GCP1_AD_DJI_0404   
1      NaN  0708_001_PM3_GCP1_AD_DJI_0405   
2      NaN  0708_001_PM3_GCP1_AD_DJI_0406   
3      NaN  0708_001_PM3_GCP1_AD_DJI_0407   

                 

In [7]:
import pandas as pd
from moviepy.video.io.VideoFileClip import VideoFileClip
from tqdm import tqdm 
import os


def convert_path(path):
    # Remove 'file://' prefix if present
    if path.startswith('file://'):
        path = path[7:]
    # Normalize the path and ensure it's in the correct format
    path = os.path.normpath(path)
    return path

# Define a function to trim the video
def trim_video(row):
    video_path = convert_path(row['HighRes_Drive_Path'].strip())  # Normalize and strip any leading/trailing whitespace
    if video_path.startswith("\\"):
        video_path = video_path[1:]  # Remove leading backslash if present
    start_time = row['start']
    end_time = row['end']
    unique_id = row['unique_id']
    
    try:
        # Load the video
        with VideoFileClip(video_path) as video:
            if start_time == "start" and pd.notnull(end_time) and end_time != "end":
                end_time = float(end_time)
                trimmed_video = video.subclip(0, end_time)
            elif end_time == "end" and pd.notnull(start_time) and start_time != "start":
                start_time = float(start_time)
                trimmed_video = video.subclip(start_time, video.duration)
            elif start_time == "start" and end_time == "end":
                trimmed_video = video
            elif pd.notnull(start_time) and pd.notnull(end_time) and start_time != "start" and end_time != "end":
                start_time = float(start_time)
                end_time = float(end_time)
                trimmed_video = video.subclip(start_time, end_time)
            else:
                raise ValueError(f"Invalid start or end time for unique ID: {unique_id}")

            # Define the output directory and filename
            output_folder = os.path.join(output_directory, unique_id)
            os.makedirs(output_folder, exist_ok=True)
            output_filename = f"{unique_id}_P01.mp4"
            output_path = os.path.join(output_folder, output_filename)
            
            # Save the trimmed video with the same quality as the original
            trimmed_video.write_videofile(output_path, codec='libx264', preset='ultrafast', audio_codec='aac', temp_audiofile='temp-audio.m4a', remove_temp=True)
            print(f"Successfully processed: {output_filename}")

    except Exception as e:
        print(f"Error processing {unique_id}: {e}")

# Function to process videos by unique IDs
def process_videos_by_ids(unique_ids):
    for unique_id in unique_ids:
        unique_id = unique_id.strip()  # Remove any leading/trailing whitespace
        # Debug statement to check the unique ID
        print(f"Processing unique ID: {unique_id}")

        # Find the row corresponding to the unique ID
        selected_row = data[data['unique_id'].astype(str).str.strip() == unique_id]
        
        if not selected_row.empty:
            video_path = selected_row['HighRes_Drive_Path'].values[0]
            print(f"Found video path: {video_path}")
            selected_row.apply(trim_video, axis=1)
        else:
            print(f"No video found with the unique ID: {unique_id}")

import random
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def extract_frames(video_path, output_folder):
    video = VideoFileClip(video_path)
    duration = int(video.duration)

    for sec in frame_indices:
        frame_time = sec + random.random()  # Random time within each second
        frame = video.get_frame(frame_time)
        frame_filename = f"{os.path.basename(video_path).split('.')[0]}_{sec+1}_{int(frame_time*1000)}.png"
        frame_path = os.path.join(output_folder, frame_filename)
        video.save_frame(frame_path, t=frame_time)
        print(f"Extracted frame at {frame_time:.2f}s to {frame_path}")

# Example usage

def process_extracted_videos(unique_ids):
    for unique_id in unique_ids:
        unique_id = unique_id.strip()  # Remove any leading/trailing whitespace
        # Define the output folder
        output_folder = os.path.join(output_directory, unique_id)
        video_filename = f"{unique_id}_P01.mp4"
        video_path = os.path.join(output_folder, video_filename)
        
        if os.path.exists(video_path):
            print(f"Processing extracted frames for video: {video_path}")
            
            #Display progress bar for frame extraction
            video = VideoFileClip(video_path)
            duration = int(video.duration)
            
            # Calculate the number of frames to extract as 1/3 of the total duration
            frames_to_extract = int(duration / 3) 
            frame_indices = sorted(random.sample(range(duration), frames_to_extract))
            
            with tqdm(total=len(frame_indices), desc="Extracting frames", unit="frame") as pbar:
                for sec in frame_indices:
                    frame_time = sec + random.random()
                    frame = video.get_frame(frame_time)
                    frame_filename = f"{os.path.basename(video_path).split('.')[0]}_{sec+1}_{int(frame_time*1000)}.png"
                    frame_path = os.path.join(output_folder, frame_filename)
                    video.save_frame(frame_path, t=frame_time)
                    print(f"Extracted frame at {frame_time:.2f}s to {frame_path}")
                    
                    # Update the progress bar after each frame extraction
                    pbar.update(1)

        else:
            print(f"No trimmed video found for unique ID: {unique_id}")


In [8]:

# output_directory = "D:\\wolf_drone_data\\cut_videos" #path on windows
output_directory = "/Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos"

# Define your list of unique IDs (copy multiple IDs) 
unique_ids = ['0708_001_PM3_GCP1_AD_DJI_0404', '0708_001_PM3_GCP1_AD_DJI_0405']


In [9]:
# # Process/ trim the videos by unique ID
process_videos_by_ids(unique_ids)

Processing unique ID: 0708_001_PM3_GCP1_AD_DJI_0404
Found video path: /Users/almazbaghirova/Desktop/Hiwi/VideoCut/videos/DJI_0404.MP4


t:  42%|████▎     | 425/1000 [00:44<00:22, 26.05it/s, now=None]

Moviepy - Building video /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01.mp4.
Moviepy - Writing video /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01.mp4



t:  42%|████▎     | 425/1000 [01:24<00:22, 26.05it/s, now=None]

Moviepy - Done !
Moviepy - video ready /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01.mp4
Successfully processed: 0708_001_PM3_GCP1_AD_DJI_0404_P01.mp4
Processing unique ID: 0708_001_PM3_GCP1_AD_DJI_0405
Found video path: /Users/almazbaghirova/Desktop/Hiwi/VideoCut/videos/DJI_0405.MP4


t:  42%|████▎     | 425/1000 [01:27<00:22, 26.05it/s, now=None]

Moviepy - Building video /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0405/0708_001_PM3_GCP1_AD_DJI_0405_P01.mp4.
Moviepy - Writing video /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0405/0708_001_PM3_GCP1_AD_DJI_0405_P01.mp4



t:  42%|████▎     | 425/1000 [02:07<00:22, 26.05it/s, now=None]

Moviepy - Done !
Moviepy - video ready /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0405/0708_001_PM3_GCP1_AD_DJI_0405_P01.mp4
Successfully processed: 0708_001_PM3_GCP1_AD_DJI_0405_P01.mp4


In [10]:
# Process the extracted frames for the trimmed videos
process_extracted_videos(unique_ids)

Processing extracted frames for video: /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01.mp4


Extracted frame at 2.36s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01_3_2355.png


Extracted frame at 4.17s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01_5_4170.png


Extracted frame at 8.23s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01_9_8232.png


Extracted frame at 11.93s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0404/0708_001_PM3_GCP1_AD_DJI_0404_P01_12_11934.png


...

Extracted frame at 9.09s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0405/0708_001_PM3_GCP1_AD_DJI_0405_P01_10_9090.png


Extracted frame at 11.30s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0405/0708_001_PM3_GCP1_AD_DJI_0405_P01_12_11298.png


Extracted frame at 14.36s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0405/0708_001_PM3_GCP1_AD_DJI_0405_P01_15_14362.png


Extracting frames: 100%|██████████| 6/6 [00:29<00:00,  4.96s/frame]

Extracted frame at 19.13s to /Users/almazbaghirova/Desktop/Hiwi/VideoCut/cut_videos/0708_001_PM3_GCP1_AD_DJI_0405/0708_001_PM3_GCP1_AD_DJI_0405_P01_20_19129.png


In [ ]:
# This function deletes the original videos after processing is completed.
def delete_original_videos(unique_ids):
    for unique_id in unique_ids:
        unique_id = unique_id.strip()
        selected_row = data[data['unique_id'].astype(str).str.strip() == unique_id]
        
        if not selected_row.empty:
            video_path = selected_row['HighRes_Drive_Path'].values[0]
            
            if os.path.exists(video_path):
                os.remove(video_path)
                print(f"Deleted original video: {video_path}")
            else:
                print(f"Original video not found: {video_path}")
        else:
            print(f"No video found with the unique ID: {unique_id}")

delete_original_videos(unique_ids)



Deleted original video: /Users/almazbaghirova/Desktop/Hiwi/VideoCut/videos/DJI_0404.MP4
Deleted original video: /Users/almazbaghirova/Desktop/Hiwi/VideoCut/videos/DJI_0405.MP4
